In [ ]:
import json
from utils import ai_generate, parse_llm_answer
import matplotlib.pyplot as plt

In [ ]:
with open("users.json", "r") as f:
    users = json.load(f)

In [ ]:
twins = ["Phelim", "Liam"]
unique_names = set([x["Name"].strip() for x in users if x["Name"] not in twins])

In [ ]:
unique_names, len(unique_names)

In [ ]:
hetero_boys = [
    "Jack Butler",
    "Jack Hussey",
    "Robin Caird",
    "Jordan Killen",
    "Jack M",
    "Davide",
]
hetero_girls = ["Robyn", "Lorna", "Sara", "Covi", "Sofia Miron Barroso"]

couples = [("Jack Butler", "Sofia"), ("Jack Hussey", "Robyn"), ("Robin Caird", "Lorna")]

In [ ]:
final_convos = []

for name in unique_names:
    all_convos = [x for x in users if name == x["Name"].strip()]
    longest_convos = sorted(all_convos, key=lambda x: len(x["AI Chat"]), reverse=True)
    print(name, [len(x["AI Chat"]) for x in longest_convos])
    final_convos.append(longest_convos[0])

In [ ]:
with open("personas.json", "r") as f:
    personas = json.load(f)
persona_names = [x["name"] for x in personas]
for name in unique_names:
    if name not in persona_names:
        convo = [x for x in final_convos if name == x["Name"].strip()][0]["AI Chat"]
        actual_convo = convo[2:]

        convo_str = "\n\n".join([f"{x['role']}: {x['content']}" for x in actual_convo])
        system_prompt = "As a Personality Distillation Representative for the world's greatest dating site, your primary responsibility is to meticulously analyze and distill the personality of dating app users based on transcripts of their conversations with other individuals within the company. The goal is to create detailed and insightful profiles that accurately reflect the user's personality, interests, and communication style. The transcript can be found below, and your task is to generate a personality profile for the user based on the conversation."
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": convo_str},
        ]

        response = ai_generate(messages)
        personas.append({"name": name, "persona": response})

with open("personas.json", "w") as f:
    json.dump(personas, f, indent=4)

all_persona_names = set([x["name"] for x in personas])

In [ ]:
with open("matches.json", "r") as f:
    matches = json.load(f)
match_tuples = [(m["person_1"], m["person_2"]) for m in matches]

for one in hetero_girls:
    for two in hetero_boys:
        if (one, two) not in match_tuples and (two, one) not in match_tuples:
            persona_one = [x for x in personas if x["name"] == one][0]
            persona_two = [x for x in personas if x["name"] == two][0]
            system_prompt = "As a Matchmaking Guru for the world's greatest dating site, your pivotal role involves delving deep into the intricacies of individual candidates' personas to make well-informed matchmaking decisions. Your mission is to honestly and objectively assess whether they are compatible based on their personalities, passions, and backgrounds. It is crucial to resist the temptation to merely please or agree with the users, as your ultimate goal is to foster genuine connections. Below, you will find two meticulously crafted personas. Your task is to provide a thoughtful and candid assessment, assigning a numerical score from 1 to 10 inside \\boxed{{answer}}. Remember, your integrity as a Matchmaking Guru is paramount. Honesty, even when faced with potential discrepancies or challenging matches, will ultimately lead to more meaningful connections for the users on our platform."

            messages = [
                {"role": "system", "content": system_prompt},
                {
                    "role": "user",
                    "content": f"Person 1:\n {persona_one['persona']}. \n\nPerson 2: {persona_two['persona']} \n\nVerdict:",
                },
            ]

            response = ai_generate(messages)
            matches.append({"person_1": one, "person_2": two, "verdict": response})

with open("matches.json", "w") as f:
    json.dump(matches, f, indent=4)

In [ ]:
for person in all_persona_names:
    all_matches = [
        x for x in matches if x["person_1"] == person or x["person_2"] == person
    ]
    not_me = [
        x["person_1"] if x["person_1"] != person else x["person_2"] for x in all_matches
    ]
    verdicts = [parse_llm_answer(x["verdict"]) for x in all_matches]
    plt.figure(figsize=(10, 5))
    for other, verdict in zip(not_me, verdicts):
        plt.bar(other, verdict)
    plt.title(f"matches for {person}")
    plt.ylim(0, 10)
    plt.show()